<!-- <img src="platia.png" width="150" height="150"> -->

<img src="https://www.cpqd.com.br/wp-content/uploads/2020/07/ico-plat1-268x300.png" width='150' height='150'>

# You Only Look Once (YOLO)

Este componente utiliza o modelo YOLO para classificação usando [Yolov4](https://pypi.org/project/yolov4/). <br>
Está é uma biblioteca que encapsula o modelo yolov4 com algumas variações, podendo utilizar o modelo completo, como também,a versão reduzida.

Este notebook apresenta:
- como usar o [SDK](https://platiagro.github.io/sdk/) para carregar datasets, salvar modelos e outros artefatos.
- como declarar parâmetros e usá-los para criar componentes reutilizáveis.

## Declaração de parâmetros e hiperparâmetros

Declare parâmetros com o botão  na barra de ferramentas.<br>
O parâmetro `dataset` identifica os conjuntos de dados. Você pode importar arquivos de dataset com o botão  na barra de ferramentas.

In [ ]:
# dataset = '/tmp/data/coco.zip'

## Extração dos dados do arquivo .zip

In [ ]:
folder = dataset.split('.')[0]

!mkdir -p {folder}
!unzip -o {dataset} -d {folder}

In [ ]:
import os

images = os.listdir(f'{folder}')

images = [x for x in images if not x.startswith('.')]

## Classificação das imagens

In [ ]:
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf

In [ ]:
!pip install gdown

In [ ]:
# Download classes
!wget https://raw.githubusercontent.com/platiagro/projects/add-yolov4/samples/default-yolo/coco.names

# Download weights
!gdown --id '15P4cYyZ2Sd876HKAEWSmeRdFl_j-0upi' --output 'yolov4.weights'

In [ ]:
# Read and format classes list
file = open('coco.names', 'r')
names = file.read().split('\n')

In [ ]:
!pip install yolov4

In [ ]:
!pip install emcee

In [ ]:
!pip install scikit-optimize

In [ ]:
!pip install pyDOE

In [ ]:
from yolov4.tf import YOLOv4

# Load the model
yolo = YOLOv4()

# Configure classes
yolo.classes = "coco.names"

# Load weights
yolo.make_model()
yolo.load_weights("yolov4.weights", weights_type="yolo")

In [ ]:
# Classify images

result = {}

for img in images:
    
    ext = img.split('.')[-1]
    
    if ext in ['jpg', 'jpeg', 'png']:
    
        frame = cv2.imread(f'{folder}/{img}')

        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        bboxes = yolo.predict(frame)

        result[img] = []

        for box in bboxes:
            result[img].append({'class': names[int(box[4])], 'probability': box[5], 'coords': box[0:4]})

## Salva modelo e resultados

In [ ]:
from joblib import dump

artifacts = {
    "names": names,
    "result": result
}

dump(artifacts, "/tmp/data/yolo.joblib")